In this notebook I applied two of the best know Deep Learning Models LSTM and CNN. I wanted to see if they can outperform the Machine Learning Algoriths I used in the previous notebook

In [37]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from keras.preprocessing import sequence
from keras.preprocessing import text
from keras.layers import Dropout, Activation
from keras.layers import Conv1D, Flatten
from keras import metrics
from keras.utils import to_categorical
from keras.layers import GlobalMaxPooling1D, Input, MaxPooling1D

In [38]:
df = pd.read_pickle("./twitter_data.pkl")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   id        30000 non-null  int64         
 1   text      30000 non-null  object        
 2   date      30000 non-null  datetime64[ns]
 3   source    30000 non-null  object        
 4   likes     30000 non-null  int64         
 5   retweets  30000 non-null  int64         
 6   class     30000 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 1.6+ MB


,id,text,date,source,likes,retweets,class
0,1288573860627877888,A medical group is warning that without action...,2020-07-29 20:35:23,TweetDeck,23,14,0
1,1288572426771554305,Covid-19 is now one of the leading causes of d...,2020-07-29 20:29:41,TweetDeck,91,47,0
2,1288570544481345538,"JUST IN: More than 150,000 people have died in...",2020-07-29 20:22:12,SocialFlow,242,145,0
3,1288566498995380232,This woman went to the hospital after breaking...,2020-07-29 20:06:08,SocialFlow,379,63,0
4,1288562923904724992,President Trump’s tweet on dismantling an Obam...,2020-07-29 19:51:56,Twitter Media Studio,473,164,0


Here I did the same thing as the previous notebook. I seperated the tweets related to the corona virus from the other tweets. In order to test the models on them while training it on the general tweets. 

In [39]:
corona_tweets = list()
for i in range(0, len(df)):
    t=df['text'][i]
    t=t.lower()
    if t.find('corona')!=-1 or t.find('covid')!=-1: 
        corona_tweets.append([df['id'][i],
                         df['text'][i],
                         df['date'][i],
                         df['source'][i],
                         df['likes'][i],
                         df['retweets'][i],
                         df['class'][i]
                         ])
        df=df.drop(i)
df.reset_index(drop=True,inplace=True)
df_ct= pd.DataFrame(corona_tweets,columns = ['id' , 'text', 'date','source','likes','retweets','class'])
df_ct.info()   
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3949 entries, 0 to 3948
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   id        3949 non-null   int64         
 1   text      3949 non-null   object        
 2   date      3949 non-null   datetime64[ns]
 3   source    3949 non-null   object        
 4   likes     3949 non-null   int64         
 5   retweets  3949 non-null   int64         
 6   class     3949 non-null   int64         
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 216.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26051 entries, 0 to 26050
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   id        26051 non-null  int64         
 1   text      26051 non-null  object        
 2   date      26051 non-null  datetime64[ns]
 3   source    26051 non-null  object        
 4   likes     26051 non-

In [40]:
#Shuffling the Data
df = df.sample(frac=1).reset_index(drop=True)
df_ct = df_ct.sample(frac=1).reset_index(drop=True)


In [41]:
y_train = df['class'].values
y_test = df_ct['class'].values


In [42]:
X_train = df['text'].values
X_test = df_ct['text'].values

In [43]:
#This class allows to vectorize a text corpus, by turning each text into either a sequence of integers 
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [44]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [50]:
#This function transforms a list of sequences into a 2D Numpy array of shape 
X_train_pad = pad_sequences(X_train_seq, maxlen=20, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=20, padding='post')


In [51]:
lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=64551, output_dim=20, input_length=20))
lstm_model.add(LSTM(400))
lstm_model.add(Dense(1, activation='sigmoid'))

lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 20, 20)            1291020   
_________________________________________________________________
lstm_3 (LSTM)                (None, 400)               673600    
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 401       
Total params: 1,965,021
Trainable params: 1,965,021
Non-trainable params: 0
_________________________________________________________________


In [52]:
history = lstm_model.fit(X_train_pad, y_train, epochs=10, batch_size=64, 
                        validation_data=(X_test_pad, y_test))

Epoch 1/10
408/408 [==============================] - 69s 170ms/step - loss: 0.3268 - accuracy: 0.8546 - val_loss: 0.2474 - val_accuracy: 0.9017
Epoch 2/10
408/408 [==============================] - 69s 169ms/step - loss: 0.1013 - accuracy: 0.9641 - val_loss: 0.2656 - val_accuracy: 0.9149
Epoch 3/10
408/408 [==============================] - 69s 169ms/step - loss: 0.0238 - accuracy: 0.9932 - val_loss: 0.3449 - val_accuracy: 0.9109
Epoch 4/10
408/408 [==============================] - 69s 170ms/step - loss: 0.0096 - accuracy: 0.9975 - val_loss: 0.3163 - val_accuracy: 0.8979
Epoch 5/10
408/408 [==============================] - 69s 169ms/step - loss: 0.0018 - accuracy: 0.9996 - val_loss: 0.5025 - val_accuracy: 0.9091
Epoch 6/10
408/408 [==============================] - 69s 169ms/step - loss: 1.3401e-04 - accuracy: 1.0000 - val_loss: 0.6929 - val_accuracy: 0.9114
Epoch 7/10
408/408 [==============================] - 69s 169ms/step - loss: 1.9094e-05 - accuracy: 1.0000 - val_loss: 0.8293 

In [53]:
acc = lstm_model.evaluate(X_test_pad, y_test)
print("Test loss is {0:.2f} accuracy is {1:.2f} ".format(acc[0],acc[1]))

124/124 [==============================] - 3s 28ms/step - loss: 1.0499 - accuracy: 0.9124
Test loss is 1.05 accuracy is 0.91 


As you can see, applying LSTM gave a test accuracy of 92%.<br>
Next I will apply CNN to see if it can give us a better accuracy

In [57]:
cnn_model = Sequential()
    # we start off with an efficient embedding layer which maps
    # our vocab indices into embedding_dims dimensions
    # 1000 is num_max
cnn_model.add(Embedding(64551, 100, input_length=1000))
cnn_model.add(Dropout(0.2))
cnn_model.add(Conv1D(64,
                     3,
                     padding='valid',
                     activation='relu',
                     strides=1))
cnn_model.add(GlobalMaxPooling1D())
cnn_model.add(Dense(256))
cnn_model.add(Dropout(0.2))
cnn_model.add(Activation('relu'))
cnn_model.add(Dense(1))
cnn_model.add(Activation('sigmoid'))
cnn_model.summary()
cnn_model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 1000, 100)         6455100   
_________________________________________________________________
dropout_8 (Dropout)          (None, 1000, 100)         0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 998, 64)           19264     
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 64)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 256)               16640     
_________________________________________________________________
dropout_9 (Dropout)          (None, 256)               0         
_________________________________________________________________
activation_8 (Activation)    (None, 256)             

In [58]:
X_train_pad = pad_sequences(X_train_seq, maxlen=1000, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=1000, padding='post')


In [59]:
history = cnn_model.fit(X_train_pad,
    y_train,
    batch_size=64,
    epochs=10,
    validation_data=(X_test_pad, y_test))

Epoch 1/10
408/408 [==============================] - 136s 332ms/step - loss: 0.3031 - acc: 0.8542 - val_loss: 0.3232 - val_acc: 0.8790
Epoch 2/10
408/408 [==============================] - 140s 343ms/step - loss: 0.0475 - acc: 0.9840 - val_loss: 0.2519 - val_acc: 0.9190
Epoch 3/10
408/408 [==============================] - 136s 333ms/step - loss: 0.0028 - acc: 0.9993 - val_loss: 0.2877 - val_acc: 0.9144
Epoch 4/10
408/408 [==============================] - 136s 332ms/step - loss: 1.2043e-04 - acc: 1.0000 - val_loss: 0.3429 - val_acc: 0.9015
Epoch 5/10
408/408 [==============================] - 136s 333ms/step - loss: 4.8127e-05 - acc: 1.0000 - val_loss: 0.3489 - val_acc: 0.9010
Epoch 6/10
408/408 [==============================] - 136s 333ms/step - loss: 3.1299e-05 - acc: 1.0000 - val_loss: 0.3661 - val_acc: 0.8969
Epoch 7/10
408/408 [==============================] - 139s 340ms/step - loss: 2.0375e-05 - acc: 1.0000 - val_loss: 0.3618 - val_acc: 0.8995
Epoch 8/10
408/408 [============

In [61]:
acc = cnn_model.evaluate(X_test_pad, y_test)
print("Test loss is {0:.2f} accuracy is {1:.2f} ".format(acc[0],acc[1]))

124/124 [==============================] - 3s 26ms/step - loss: 0.3971 - acc: 0.8924
Test loss is 0.40 accuracy is 0.89 
